In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python3)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

-- The C compiler identification is AppleClang 10.0.1.10010046
-- The CXX compiler identification is AppleClang 10.0.1.10010046
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
CMake Error at CMakeLists.txt:26 (add_subdirectory):
  add_subdirectory given source "pybind11" which is not an existing
  directory.


CMake Error at CMakeLists.txt:28 (pybind11_add_module):
  Unknown CMake command "py

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: No se puede efectuar open: No existe el archivo o el directorio
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [3]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

# Experimento KNN SIN PCA


In [5]:
import sentiment
from sklearn.metrics import accuracy_score
import time


scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))

for k in k_values:
    clf = sentiment.KNNClassifier(k)
    
    clf.fit(X_train, y_train)
    start = time.time()
    y_pred = clf.predict(X_test)
    end = time.time()
    
    acc = accuracy_score(y_test, y_pred)
    difTime = end - start
    print("k: {}   acc: {} time: {}".format(k, acc, difTime))
    scores.append(acc)
    
print(scores)

k: 1   acc: 0.5499601593625498 time: 51.06967377662659
k: 5   acc: 0.5915537848605578 time: 49.47508239746094
k: 10   acc: 0.6065338645418327 time: 50.47711181640625
k: 15   acc: 0.6237450199203187 time: 49.1464467048645
k: 20   acc: 0.6341035856573706 time: 50.39867854118347
k: 25   acc: 0.6341035856573706 time: 50.54355716705322
k: 50   acc: 0.6473306772908366 time: 49.5083794593811
k: 100   acc: 0.6388844621513944 time: 49.11234951019287
k: 150   acc: 0.6283665338645419 time: 49.75154447555542
k: 200   acc: 0.6224701195219123 time: 49.04815745353699
k: 250   acc: 0.6224701195219123 time: 48.28087139129639
k: 300   acc: 0.6167330677290837 time: 49.11891770362854
k: 350   acc: 0.6162549800796813 time: 49.89825415611267
[0.5499601593625498, 0.5915537848605578, 0.6065338645418327, 0.6237450199203187, 0.6341035856573706, 0.6341035856573706, 0.6473306772908366, 0.6388844621513944, 0.6283665338645419, 0.6224701195219123, 0.6224701195219123, 0.6167330677290837, 0.6162549800796813]


# Experimento KNN variando max y min df

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import sentiment
from sklearn.metrics import accuracy_score

maxDF_values = [0.80,0.85,0.90,0.92,0.95,1.00]
minDF_values = [0.01,0.02,0.03,0.05,0.08,0.10]

for maxDF in maxDF_values:
    
    for minDF in minDF_values:
        
        vectorizer = CountVectorizer(max_df=maxDF, min_df=minDF, max_features=5000)

        vectorizer.fit(text_train)

        X_train1, y_train1 = vectorizer.transform(text_train), (label_train == 'pos').values
        X_test1, y_test1 = vectorizer.transform(text_test), (label_test == 'pos').values

        scores = []
        k_values = [50]

        for k in k_values:
            clf = sentiment.KNNClassifier(k)
            clf.fit(X_train1, y_train1)
            y_pred1 = clf.predict(X_test1)
            acc = accuracy_score(y_test1, y_pred1)
            print("k: {}   acc: {} maxDF: {} minDF: {}".format(k, acc,maxDF, minDF))
            scores.append(acc)
    
print(scores)

k: 50   acc: 0.6643824701195219 maxDF: 0.8 minDF: 0.01
k: 50   acc: 0.6624701195219124 maxDF: 0.8 minDF: 0.02
k: 50   acc: 0.6616733067729084 maxDF: 0.8 minDF: 0.03
k: 50   acc: 0.6661354581673307 maxDF: 0.8 minDF: 0.05
k: 50   acc: 0.6541832669322709 maxDF: 0.8 minDF: 0.08
k: 50   acc: 0.6567330677290837 maxDF: 0.8 minDF: 0.1
k: 50   acc: 0.6621513944223107 maxDF: 0.85 minDF: 0.01
k: 50   acc: 0.6613545816733067 maxDF: 0.85 minDF: 0.02
k: 50   acc: 0.660398406374502 maxDF: 0.85 minDF: 0.03
k: 50   acc: 0.6599203187250996 maxDF: 0.85 minDF: 0.05
k: 50   acc: 0.6535458167330678 maxDF: 0.85 minDF: 0.08
k: 50   acc: 0.6513147410358566 maxDF: 0.85 minDF: 0.1
k: 50   acc: 0.6473306772908366 maxDF: 0.9 minDF: 0.01
k: 50   acc: 0.6438247011952192 maxDF: 0.9 minDF: 0.02
k: 50   acc: 0.6441434262948207 maxDF: 0.9 minDF: 0.03
k: 50   acc: 0.6334661354581673 maxDF: 0.9 minDF: 0.05
k: 50   acc: 0.6326693227091633 maxDF: 0.9 minDF: 0.08
k: 50   acc: 0.6310756972111554 maxDF: 0.9 minDF: 0.1
k: 50   

# Experimento KNN CON PCA

In [7]:
import sentiment
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.8, min_df=0.05, max_features=5000)

vectorizer.fit(text_train)

X_train2, y_train2 = vectorizer.transform(text_train), (label_train == 'pos').values
X_test2, y_test2 = vectorizer.transform(text_test), (label_test == 'pos').values

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))
alpha_values = [50,100,150]

for alpha in alpha_values:
    for k in k_values:
        X_train2aux = X_train2
        X_test2aux = X_test2
        y_train2aux = y_train2
        y_test2aux = y_test2
        
        pca = sentiment.PCA(alpha)
        pca.fit(X_train2aux.toarray())
        
        X_train2aux = pca.transform(X_train2aux)
        X_test2aux = pca.transform(X_test2aux)
        
        clf = sentiment.KNNClassifier(k)
        clf.fit(X_train2aux, y_train2aux)
        start = time.time()
        y_pred2 = clf.predict(X_test2aux)
        end = time.time()
        difTime = end - start
        acc = accuracy_score(y_test2aux, y_pred2)
        print("k: {}  acc: {} Alpha: {} time: {}".format(k, acc, alpha, difTime))
        scores.append(acc)

print(scores)

k: 1  acc: 0.5773705179282869 Alpha: 50 time: 5.169682502746582
k: 5  acc: 0.6138645418326694 Alpha: 50 time: 5.11849308013916
k: 10  acc: 0.6240637450199203 Alpha: 50 time: 5.113521099090576
k: 15  acc: 0.6455776892430279 Alpha: 50 time: 5.121351718902588
k: 20  acc: 0.6545019920318725 Alpha: 50 time: 5.117036581039429
k: 25  acc: 0.6581673306772908 Alpha: 50 time: 5.117199897766113
k: 50  acc: 0.6667729083665339 Alpha: 50 time: 5.121857643127441
k: 100  acc: 0.6635856573705179 Alpha: 50 time: 5.132898569107056
k: 150  acc: 0.6545019920318725 Alpha: 50 time: 5.135848522186279
k: 200  acc: 0.6564143426294821 Alpha: 50 time: 5.132761716842651
k: 250  acc: 0.6525896414342629 Alpha: 50 time: 5.139922380447388
k: 300  acc: 0.6479681274900398 Alpha: 50 time: 5.143077373504639
k: 350  acc: 0.6495617529880479 Alpha: 50 time: 5.153941631317139
k: 1  acc: 0.5711553784860558 Alpha: 100 time: 6.971991300582886
k: 5  acc: 0.6199203187250996 Alpha: 100 time: 6.9746010303497314
k: 10  acc: 0.6353784

# Variando cantidad de instancias de entrenamiento KNN SIN PCA

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
import sentiment
from sklearn.metrics import accuracy_score
import time

vectorizer = CountVectorizer(max_df=0.8, min_df=0.05, max_features=5000)

vectorizer.fit(text_train)

X_train3, y_train3 = vectorizer.transform(text_train), (label_train == 'pos').values
X_test3, y_test3 = vectorizer.transform(text_test), (label_test == 'pos').values

x_values = [5,10,20,50,100,500,1000,2000,3000,4000,5000,6500]

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))

for x in x_values:
    x_train3Aux = X_train3
    y_train3Aux = y_train3
    x_train3Aux = x_train3Aux[0:x]
    y_train3Aux = y_train3Aux[0:x]
        
    for k in k_values:
        if k >= x:
            break
        x_train3Aux = X_train3
        y_train3Aux = y_train3
        x_train3Aux = x_train3Aux[0:x]
        y_train3Aux = y_train3Aux[0:x]

        clf = sentiment.KNNClassifier(k)

        clf.fit(x_train3Aux, y_train3Aux)
        start = time.time()
        y_pred3 = clf.predict(X_test3)
        end = time.time()
        difTime = end - start
        acc = accuracy_score(y_test3, y_pred3)
        print("k: {}   acc: {} instancias: {} time: {}".format(k, acc, x, difTime))
        scores.append(acc)
    
print(scores)

k: 1   acc: 0.5034262948207171 instancias: 5 time: 0.7675349712371826
k: 1   acc: 0.506613545816733 instancias: 10 time: 0.7734503746032715
k: 5   acc: 0.4952988047808765 instancias: 10 time: 0.7764861583709717
k: 1   acc: 0.5177689243027889 instancias: 20 time: 0.7907943725585938
k: 5   acc: 0.507410358565737 instancias: 20 time: 0.7925739288330078
k: 10   acc: 0.4970517928286853 instancias: 20 time: 0.791630744934082
k: 15   acc: 0.4964143426294821 instancias: 20 time: 0.7923545837402344
k: 1   acc: 0.5225498007968128 instancias: 50 time: 0.8378913402557373
k: 5   acc: 0.5397609561752988 instancias: 50 time: 0.8402295112609863
k: 10   acc: 0.5311553784860558 instancias: 50 time: 0.8468739986419678
k: 15   acc: 0.5144223107569721 instancias: 50 time: 0.8416399955749512
k: 20   acc: 0.5150597609561753 instancias: 50 time: 0.841062068939209
k: 25   acc: 0.5048605577689244 instancias: 50 time: 0.8430542945861816
k: 1   acc: 0.5391235059760956 instancias: 100 time: 0.9229171276092529
k: 5

# Variando cantidad de instancias de entrenamiento KNN CON PCA

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import sentiment
from sklearn.metrics import accuracy_score

vectorizer = CountVectorizer(max_df=0.8, min_df=0.05, max_features=5000)

vectorizer.fit(text_train)

X_train4, y_train4 = vectorizer.transform(text_train), (label_train == 'pos').values
X_test4, y_test4 = vectorizer.transform(text_test), (label_test == 'pos').values

x_values = [5,10,20,50,100,500,1000,2000,3000,4000,5000,6500]

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))
alpha_values = [50,100,150]
for x in x_values:
    x_train4Aux = X_train4
    y_train4Aux = y_train4    
    x_train4Aux = x_train4Aux[0:x]
    y_train4Aux = y_train4Aux[0:x]
    for alpha in alpha_values:
        for k in k_values:
            if k >= x:
                break
            x_train4Aux = X_train4
            y_train4Aux = y_train4
            x_train4Aux = x_train4Aux[0:x]
            y_train4Aux = y_train4Aux[0:x]
            X_test4aux = X_test4
            y_test4aux = y_test4

            pca = sentiment.PCA(alpha)
            pca.fit(x_train4Aux.toarray())

            x_train4Aux = pca.transform(x_train4Aux)
            X_test4aux = pca.transform(X_test4aux)

            clf = sentiment.KNNClassifier(k)
            clf.fit(x_train4Aux, y_train4Aux)
            start = time.time()
            y_pred4 = clf.predict(X_test4aux)
            end = time.time()
            difTime = end - start
            acc = accuracy_score(y_test4aux, y_pred4)
        
            print("k: {}   acc: {}  Alpha: {} instancias: {} time: {}".format(k, acc, alpha, x, difTime))
            scores.append(acc)

print(scores)

k: 1   acc: 0.48908366533864545  Alpha: 50 instancias: 5 time: 0.571319580078125
k: 1   acc: 0.5064541832669323  Alpha: 100 instancias: 5 time: 1.145611047744751
k: 1   acc: 0.4895617529880478  Alpha: 150 instancias: 5 time: 1.7570407390594482
k: 1   acc: 0.4952988047808765  Alpha: 50 instancias: 10 time: 0.5731074810028076
k: 5   acc: 0.5069322709163346  Alpha: 50 instancias: 10 time: 0.5719118118286133
k: 1   acc: 0.5026294820717131  Alpha: 100 instancias: 10 time: 1.1449966430664062
k: 5   acc: 0.4897211155378486  Alpha: 100 instancias: 10 time: 1.1461577415466309
k: 1   acc: 0.52  Alpha: 150 instancias: 10 time: 1.759570598602295
k: 5   acc: 0.4952988047808765  Alpha: 150 instancias: 10 time: 1.7579891681671143
k: 1   acc: 0.5118725099601593  Alpha: 50 instancias: 20 time: 0.5745441913604736
k: 5   acc: 0.5110756972111554  Alpha: 50 instancias: 20 time: 0.5749328136444092
k: 10   acc: 0.4951394422310757  Alpha: 50 instancias: 20 time: 0.5765626430511475
k: 15   acc: 0.4956175298804